# Wide & Deep Learning for Recommender Systems
- Wide Component와 Deep Component의 장점을 합친 추천 모델
- Wide model
    - memorization(기억)
    - ex) 이전에 먹었던 음식이 '햄버거(bugers)' 이면 불고기버거를 추천해주는 모델. 하지만 같은 패스트푸드인 피자를 추천하지는 못한다.
- Deep model
    - generalization
    - query를 카테고리화 시켜서 그 범주 내에 있는 item을 추천함
    - ex) 위의 예제에서 유저가 햄버거를 클릭, 검색 시 피자를 추천할 수 있음. but, over-generalization 되는 경우가 발생함
    - over-generalization
        - '따뜻한 아메리카노' 검색 시 '커피'라는 일반화 범주에서 '아이스 라떼'를 추천 할 수 있다는 것. 
        - 유저는 따뜻한 음료를 원하고 라떼를 싫어할 수 있지만 전혀 다른 결과를 예측할 수 있는 가능성이 있음.

## References
- [구글에서 발표한 논문](https://arxiv.org/pdf/1606.07792.pdf)
- [텐서플로우 튜토리얼](https://tensorflowkorea.gitbooks.io/tensorflow-kr/content/g3doc/tutorials/wide_and_deep/)
- [참고 블로그](https://bcho.tistory.com/tag/wide%20and%20deep%20model)

## Wide & Deep 모델이란?
- linear 모델의 장점과 neural network(신경망)의 장점을 합치는 모델이다.
- 즉, 선형 모델의 저장법(memorization)과 신경망의 일반화(generalization)의 강점을 합친 모델이다.
- 이 방법은 sparse 입력 feature의 regression 문제와 classification 문제에 유용하다.

### 모델의 3단계 프로세스
1. wide 부분에 대한 features 선택
    - 사용하길 원하는 sparse base column 과 crossed column 들을 선택한다
2. deep 부분에 대한 features 선택 
    - continuous feature(연속된 열), categorical feature(분류 열)의 embedding dimension, 그리고 hidden layer 크기를 선택한다.
3. 이들을 Wide & Deep 모델에 적용한다.